In [11]:
import krippendorff
import numpy as np
import json     

In [12]:
def load_annotations(jsonl_path):
    scores = []
    with open(jsonl_path, 'r') as f:
        for line in f:
            obj = json.loads(line)
            row = obj.get("scores")
            cleaned = [int(x) if x is not None else None for x in row]
            if cleaned:  # ensure it’s not empty
                scores.append(cleaned)
    return scores

In [13]:
# Transpose data to shape (annotator, item)
def compute_krippendorff_alpha(annotations):
    data = np.array(annotations).T
    alpha = krippendorff.alpha(reliability_data=data, level_of_measurement='interval')
    return alpha

In [14]:
jsonl_path = "annotations__educational_content__en__gt.jsonl"
annotations = load_annotations(jsonl_path)
alpha = compute_krippendorff_alpha(annotations)
print(f"Krippendorff’s alpha (ordinal): {alpha:.3f}")

Krippendorff’s alpha (ordinal): 0.569
